In [1]:
# -*- coding: utf-8 -*-
#Initial Imports
import gzip#; print('gzip version: ', gzip.__version__)
import os#; print('os version: ', os.__version__)
import re#; print('re version: ', re.__version__)
import shutil#; print('shutil version: ', shutil.__version__)
import requests#; print('requests version: ', requests.__version__)
import wget#; print('wget version: ', wget.__version__)
import subprocess#; print('subprocess version: ', subprocess.__version__)
import time#; print('time version: ', time.__version__)
from datetime import datetime
from subprocess import PIPE, Popen
from fake_useragent import UserAgent
import os.path
from os import path
from Bio.ExPASy import Enzyme
import numpy as np 

### Creating Unitprot.fasta for Use in Diamond

In [ ]:
def tsv_to_fasta():
    reference_library = 'uniprot.tsv' 
    #this creates a file named 'uniprot.tsv' on the working directory. Should show within the EcoDr/EcoDr folder. 
    ua = UserAgent()
    header = {'User-Agent': str(ua.chrome)}
    uniprot_url = 'https://rest.uniprot.org/uniprotkb/stream?fields=accession%2Cec%2Csequence&format=tsv&query=%28%28ec%3A*%29%29+AND+%28reviewed%3Atrue%29'
    time.sleep(4)
    uniprot = requests.get(uniprot_url, headers=header)
    if uniprot.status_code == 200:
        with open(reference_library, 'w+') as reference_library:
            reference_library.write(uniprot.text)
    print('Uniprot Reference File Has Been Created')
####this step takes it the initial tsv and converts it to FASTA
    input = os.path.abspath('uniprot.tsv')
    output = 'uniprot.fasta'
    with open(input, 'r') as input_file, open(output, 'w') as output_file:
        header=next(input_file)
        for line in input_file:
            carrot = f'>{line}'
            new_row = re.sub(r'(.*?\t.*?)\t', r'\1\n', carrot, 1)
            new_row_with_tab_replaced_by_question_mark = new_row.replace("\t", "?")
            # Be careful with the symbols used for replacement, as they have to align with the genomic summary string parsing
            output_row_with_space_replaced_with_ampersand = new_row_with_tab_replaced_by_question_mark.replace("; ", ";_")
            #new_row2 = re.sub(r'(.*?\t.*?)\t', r'$', new_row, 0)
            #this is regex, for the record. 
            output_file.write(output_row_with_space_replaced_with_ampersand)
    print('FASTA has been created from TSV and is named', output)

### Diamond

In [ ]:
def diamond_impl(dest, name):
    print(os.getcwd())
    matches = ''
    output_folder = dest 
    print("DIAMOND search library is: ", output_folder)
    if os.path.isfile('reference.dmnd'):
        print("Library Detected")
    # If not present, then creates a DIAMOND library by referencing the exact location where the Uniprot library is saved
    # If there currently is no reference library (.dmnd), then command makedb creates a DIAMOND library
    else:
        print("Creation of DIAMOND-formatted library...")
        makedb = ['diamond', 'makedb', '--in', '/projects/jodo9280/EcoDr/EcoDr/uniprot.fasta', '-d',
                  'Uniprot_Reference_Library.dmnd']  # Reference library full pathway
        #This is a list for the DIAMOND specific makedb function. 
        subprocess.run(makedb)
        #This simply runs the function makedb
        print("Library complete")
##_______________________________________________________________# This portion does the matching. The above portion creates the reference library from the unitprot fasta
    for item in os.listdir(dest):
        # Checks for file extension
        if item.endswith('.faa') and not os.path.isfile(
                os.path.basename(os.path.abspath(item)).rsplit('.', 1)[0] + "_matches.tsv"):
            # Finds path of file
            file_path = os.path.abspath(item)
            # Finds the GCF/ASM name of the file by looking at the first part of the name before the .faa notation
            if name == "":
                print(os.path.basename(file_path).rsplit('.',1))
                file_name = (os.path.basename(file_path)).rsplit('.', 1)[0]
            else:
                file_name = name
            # New filename that ends with matches
            matches = file_name + "_matches.tsv"
            print(matches)
            # If genome has not already undergone DIAMOND search and is currently located in the correct folder, then
            # the subprocess function will run the diamond search
            if not os.path.isfile(dest + "/" + matches) and os.path.abspath(matches) != output_folder:
                print("Processing ", file_name)
                # DIAMOND search using the full pathway of the protein files, max target sequence outputs only one best
                # match with highest e-value which represent the chance of obtaining a better random match in the same database (Buchfink et al, 2021)
                blastp = ['diamond', 'blastp', '-d', 'Uniprot_Reference_Library.dmnd', '-q', file_path, '-o', matches,
                          '--max-target-seqs', '1', '--outfmt', '6']
                time.sleep(4)
                subprocess.run(blastp)
        # (2) Creates a folder for DIAMOND outputs
        #if not os.path.exists(output_folder):
            #os.makedirs('DIAMOND_matches')

    # Moves all DIAMOND search outputs into the folder
        if item.endswith('_matches.tsv'):
            if os.path.exists(os.path.join(output_folder, item)):
                print(f"Overwriting: {item}")
                os.remove(os.path.join(output_folder, item))
            shutil.move(os.path.abspath(item), output_folder)
    print("diamond_impl--success")
    # Returns the location of the DIAMOND matches folder
    return output_folder

### Calling Script

In [ ]:
metagenome = 'Some.faa metagenome file in an EnCen Directory' 
abspath = os.path.abspath(metagenome) 
metagenome_name = 'metagenome_analysis_output'
desired_location = '/projects/jodo9280/EcoDr/EcoDr/EnCen'
#______________________________________________________________#
os.chdir(desired_location) #-> we are in the folder we want
if os.path.exists(metagenome_name):
    shutil.rmtree(metagenome_name)
    os.mkdir(metagenome_name)
else:#makes a new directory called metagenome_name
    os.mkdir(metagenome_name)
desired_location = desired_location + "/" + metagenome_name #-> this will make a folder called 'metagenome_analysis_output' within the 'desired_location' 

shutil.copy(abspath, desired_location) #moves synbio file to Test Cases folder
#matches = metagenome + "_matches"

In [ ]:
diamond = diamond_impl(desired_location, metagenome) #-> Takes in the path and directory
#desired_location is where all the DIAMOND matches will go. 
